In [4]:
import logging
import os

import torch
import numpy as np
import matplotlib.pyplot as plt

import pyro
import pyro.distributions as dist

from pyro.infer import MCMC, NUTS, Predictive
from pyro.infer.mcmc.util import initialize_model, summary

In [2]:
smoke_test = ('CI' in os.environ)
assert pyro.__version__.startswith('1.8.4')

pyro.enable_validation(True)
pyro.set_rng_seed(1)
logging.basicConfig(format='%(message)s', level=logging.INFO)

# Set matplotlib settings
%matplotlib inline
plt.style.use('default')

From [here](https://docs.pyro.ai/en/stable/mcmc.html)

In [21]:
# Literal toy data generation with "true" coefficients.
true_coefs = torch.tensor([1., 2., 3.])
data       = torch.randn(2000, 3)
dim        = 3
labels     = dist.Bernoulli(logits=(true_coefs * data).sum(-1)).sample()

# Model for the data.
def model(data, labels):
    coefs = pyro.sample("beta", dist.Normal(torch.zeros(dim), torch.ones(dim)))
    # when labels are provided, y = labels. If labels is None, then dist.Bernoulli...
    # is returned. TODO: understand how the samplers use this design. 
    y = pyro.sample("y", dist.Bernoulli(logits=(coefs * data).sum(-1)), obs=labels)

nuts_kernel = NUTS(model, adapt_step_size=True)
mcmc = MCMC(nuts_kernel, num_samples=500, warmup_steps=300)
mcmc.run(data, labels)
mcmc.get_samples()["beta"].mean(0)


Sample: 100%|██████████| 800/800 [00:01, 402.94it/s, step size=6.58e-01, acc. prob=0.900]


tensor([0.9411, 1.8397, 2.7934])